## 建模部分
这里我们直接使用catboost这种快捷工具来进行。

In [1]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
import pickle

In [2]:
# 加载训练数据和测试数据
with open('data/final_data.pk', 'rb') as f:
    train_group_x, train_group_y, test_group_x, concat_feature_names = pickle.load(f)

In [3]:
# 将特征组中的离散型特征挑选出来
cate_columns = [i[0] for i in concat_feature_names if i[1] == 'i']
# 将离散型特征进行类型转换
for c in cate_columns:
    train_group_x[c] = train_group_x[c].fillna(-1).astype(int)
    test_group_x[c] = test_group_x[c].fillna(-1).astype(int)
for c in cate_columns[2:]:
    train_group_x = train_group_x.drop(c, axis=1)
    test_group_x = test_group_x.drop(c, axis=1)

In [4]:
cate_columns = cate_columns[:2]

In [5]:
train_group_x.head()

,用户年龄,用户性别,用户点击数量,用户加购数量,用户购买数量,用户收藏数量,用户点击转化率,用户加购转化率,用户收藏转化率,用户周计数活跃数量-0,...,交互-双11购买总数,交互-双11收藏总数,交互-商品数量,交互-品类数量,交互-品牌数量,交互-品类重叠系数,交互-品牌重叠系数,交互-购买主品类标签,交互-购买主品牌标签,交互-初始交互时间差
0,6,0,410.0,0.0,34.0,7.0,12.058824,0.0,0.205882,38.0,...,1.0,0.0,20.0,6.0,1.0,0.333333,0.050000,1.0,1.0,2.0
1,6,0,410.0,0.0,34.0,7.0,12.058824,0.0,0.205882,38.0,...,1.0,0.0,1.0,1.0,1.0,0.069767,0.050000,1.0,1.0,2.0
2,6,0,410.0,0.0,34.0,7.0,12.058824,0.0,0.205882,38.0,...,6.0,0.0,2.0,1.0,1.0,0.129032,0.050000,1.0,1.0,2.0
3,6,0,410.0,0.0,34.0,7.0,12.058824,0.0,0.205882,38.0,...,1.0,0.0,1.0,1.0,1.0,0.136364,0.000000,1.0,0.0,1.0
4,0,0,47.0,0.0,7.0,0.0,6.714286,0.0,0.000000,10.0,...,1.0,0.0,1.0,1.0,1.0,0.257143,0.052632,1.0,1.0,1.0


In [6]:
test_group_x.head()

,用户年龄,用户性别,用户点击数量,用户加购数量,用户购买数量,用户收藏数量,用户点击转化率,用户加购转化率,用户收藏转化率,用户周计数活跃数量-0,...,交互-双11购买总数,交互-双11收藏总数,交互-商品数量,交互-品类数量,交互-品牌数量,交互-品类重叠系数,交互-品牌重叠系数,交互-购买主品类标签,交互-购买主品牌标签,交互-初始交互时间差
0,0,0,63.0,0.0,16.0,2.0,3.937500,0.0,0.125,11.0,...,1.0,0.0,1.0,1.0,1.0,0.136364,0.05,1.0,1.0,1.0
1,2,2,71.0,0.0,6.0,0.0,11.833333,0.0,0.000,0.0,...,5.0,0.0,9.0,4.0,1.0,0.166667,0.05,1.0,1.0,1.0
2,6,0,51.0,0.0,5.0,0.0,10.200000,0.0,0.000,8.0,...,1.0,0.0,1.0,1.0,1.0,0.025641,0.05,1.0,1.0,1.0
3,6,0,51.0,0.0,5.0,0.0,10.200000,0.0,0.000,8.0,...,1.0,0.0,1.0,1.0,1.0,0.147059,0.05,1.0,1.0,1.0
4,2,1,162.0,0.0,7.0,7.0,23.142857,0.0,1.000,27.0,...,1.0,0.0,8.0,4.0,2.0,0.064516,0.10,1.0,1.0,1.0


In [7]:
train_group_y.head()

0    0
1    0
2    1
3    0
4    0
Name: label, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
# 进行训练集和验证集的拆分
x_to_train, x_to_valid, y_to_train, y_to_valid = train_test_split(train_group_x, train_group_y, test_size=0.1, random_state=502)

# 构建CatBoost分类器
cb_params = {
        'iterations': 1000,
        'loss_function': 'CrossEntropy',
        'eval_metric': 'AUC',
        'learning_rate': 0.01,
        'depth': 15,
        'use_best_model': True,
        'subsample': 0.7,
        'bootstrap_type': 'Bernoulli',
        'reg_lambda': 1,
        #'task_type': "GPU",
        #'devices': '0:1',
        'cat_features': cate_columns
    }
cat_model = CatBoostClassifier(**cb_params)
cat_model.fit(x_to_train, y_to_train, eval_set=(x_to_valid, y_to_valid), early_stopping_rounds=10, verbose=20)

0:	test: 0.6325252	best: 0.6325252 (0)	total: 7.97s	remaining: 2h 12m 45s
20:	test: 0.6995095	best: 0.6997988 (19)	total: 2m 15s	remaining: 1h 45m 10s
